In [ ]:
!python3 -m pip install --upgrade openai

In [2]:
import os
import openai
import pretty_midi
import warnings
import constants
import time
import re
openai.api_key = constants.GPT_API_KEY

In [8]:
dataset_folder = '/Users/alan/Downloads/clean_midi'

In [63]:
def get_caption(song_name, artist_name, instruments, key, bpm):
    prompt = f"Write an informative musical description of a MIDI song cover of the song \"{song_name}\" by the artist \"{artist_name}\". \
               The instrumentation of this cover includes \"{instruments}\", the key signature is {key}, and the beats per minute (BPM) is {bpm}. \
               Be as specific as possible with musical features, such as genre, texture, timbre, form, rhythm, melody, harmony, and song structure. \
               Focus especially on any highly distinctive aspects of this song. \
               This description should be roughly 200 words long and should have no conclusion paragraph."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": prompt}
            ]
    )
    caption = response['choices'][0]['message']['content'].lstrip('\n')

    return caption

def get_metadata(midi_path):
    with warnings.catch_warnings(record=True) as w:
        # set all warnings to always be triggered
        warnings.simplefilter("always")
        try:
            midi = pretty_midi.PrettyMIDI(midi_path)

            if len(w) == 0:  # no warnings means not track is not empty and no issues
                assert len(midi.key_signature_changes) == 1
                key = pretty_midi.key_number_to_key_name(midi.key_signature_changes[0].key_number)

                assert len(midi.get_tempo_changes()) == 2
                bpm = round(midi.get_tempo_changes()[1][0])

                assert len(midi.instruments) != 0
                instruments = ', '.join(list(set([instrument.name.strip() for instrument in midi.instruments])))
                print(instruments)

                return (key, bpm, instruments)
            else:
                print(f"{midi_path} had warnings while loading, skipping")
                return -1
            
        except Exception as e:
            print(e)
            return -1

def contains_number_pattern(string):
    pattern = r"\.\d\."
    return bool(re.search(pattern, string))

In [64]:
def caption_all_artist_folders(overwrite=False):
    artist_list = os.listdir(dataset_folder)
    for artist in artist_list:
        if artist.startswith('.') or not os.path.isdir(f'{dataset_folder}/{artist}'):
            continue
        
        caption_artist_folder(artist, overwrite)


def caption_artist_folder(artist, overwrite=False):
    '''
    Takes in the name of a artist folder containing .mid files and generates rich text captions
    for each song in the artist folder in .txt format

            Parameters:
                    artist (str): the artist whose songs to caption
                    overwrite (bool): specify whether captioning should overwrite existing 
                                      .txt files with the same file name
    '''
    artist_path = f'{dataset_folder}/{artist}'

    song_list = os.listdir(artist_path)
    num_songs = len(song_list)

    for i, song_filename in enumerate(song_list):
        if song_filename.startswith('.') or contains_number_pattern(song_filename):
            continue

        song_name = ''.join(song_filename.split('.')[:-1])
        caption_filename = song_name + '_caption.txt'
        
        if not overwrite and os.path.isfile(f'{artist_path}/{caption_filename}'):
            continue

        song_path = f'{artist_path}/{song_filename}'
        caption_path = f'{artist_path}/{caption_filename}'

        metadata_result = get_metadata(song_path)
        if metadata_result == -1:
            continue
        key, bpm, instruments = metadata_result

        caption = get_caption(song_name, artist, key, bpm, instruments)
        assert(len(caption) != 0)

        with open(caption_path, 'w') as f:
            f.write(caption)
        print(caption)
        print(f"{i + 1}/{num_songs} files in the {artist} folder")

        # time.sleep(12)

    print(f"Finished writing captions for {artist}.")

In [ ]:
#caption_all_artist_folders()
caption_artist_folder('Earth, Wind & Fire')